In [1]:
# 所有的头

In [2]:
import torch # for save
import os
from van import Fan
from localconfig import config
import datetime
import random
from pprint import pprint
import html
from pyltp import SentenceSplitter
from pyltp import Postagger
from pyltp import Segmentor
from pyltp import NamedEntityRecognizer
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import Image
from IPython.core.display import HTML 
import pyecharts
from pyecharts import Pie
from pyecharts import Line
import reverse_geocoder as rg

%matplotlib inline

plt.rcParams['font.sans-serif']=['FangSong'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号


In [3]:
# 所有的配置

In [4]:
keyword = '微信' # 关注的关键词
until_year = 2000 # 最早的年份
until_num = 10000 # 截止的饭否数量
timeout = (60,60) # 超时时间

In [5]:
# 登录
fan = Fan(config['Consumer key'], config['Consumer secret'])
fan.xauth(config['my username'], config['my password'])

{'oauth_token': '1405515-071ad3666b5af9441d5cbb0dc1d3985b',
 'oauth_token_secret': '43cc293f5d127423f960b8636688577b'}

In [6]:
# 本小时内可用API次数
fan.request('GET', 'account/rate_limit_status', {}, timeout=timeout)

{'reset_time': 'Mon Mar 18 23:05:05 +0000 2019',
 'remaining_hits': 150,
 'hourly_limit': 150,
 'reset_time_in_seconds': 1552950305}

In [7]:
# 采集关键词消息
def datetimefromstr(s):
    return datetime.datetime.strptime(s, '%a %b %d %H:%M:%S %z %Y')
def get_status(keyword, until_year, until_num):
    latest = fan.request('GET', 'search/public_timeline', {'q':keyword, 'count':60}, timeout=timeout)
    
    while True:
        year = datetimefromstr(latest[-1]['created_at']).year
        if len(latest) >= until_num:
            break
        if year < until_year:
            break
        print(len(latest), year,'/', until_num, until_year, end='\r')
        len_ = len(latest)
        latest += fan.request('GET', 'search/public_timeline', { 'q':keyword, 'count':60, 'max_id':latest[-1]['id']}, timeout=timeout)[1:]
        if len(latest) == len_:
            break
    return latest

In [8]:
# 下载并暂存
statuses = get_status(keyword, until_year, until_num)

In [9]:
torch.save(statuses,'tmp_statuses.torch.pkl')

In [10]:
# 恢复
# statuses = torch.load('tmp_statuses.torch.pkl')

In [11]:
print(len(statuses))

10031


In [12]:
# common
def get_top(l, num=5, reverse=False): # 从大到小
    cnt = {}
    for x in l:
        if type(x) == tuple and len(x) == 3:
            x = x[0]
        if x not in cnt:
            cnt[x] = 0
        cnt[x] += 1
    cnt_list = [(k,cnt[k],cnt[k]/len(l)) for k in cnt]
    cnt_list = sorted(cnt_list, key=lambda x:x[1])[::-1]
    if reverse:
        cnt_list = cnt_list[::-1]
    if num == -1:
        return cnt_list
    return cnt_list[:num]
def huabing(y): # x [(label,,0.xxx)]
    attr = [x[0] for x in y]
    v1 = [x[2] for x in y]
    if sum(v1) < 1.0:
        attr.append('其他')
        v1.append(1.0 - sum(v1))
    pie = Pie('')
    pie.add('', attr, v1, is_label_show=True)
    return pie

In [13]:
# 时间段分布

In [14]:
print(datetimefromstr(statuses[-1]['created_at']))
print(datetimefromstr(statuses[0]['created_at']))
begin_year = datetimefromstr(statuses[-1]['created_at']).year
end_year = datetimefromstr(statuses[0]['created_at']).year

2018-12-04 01:57:12+00:00
2019-03-18 21:48:15+00:00


In [15]:
huabing(get_top([datetimefromstr(x['created_at']).year for x in statuses], num = -1))

In [16]:
arr = sorted(get_top([datetimefromstr(x['created_at']).year for x in statuses],num=-1),key = lambda x:x[0])
attr = [str(x[0]) for x in arr ]
v1 = [x[1] for x in arr ]
line = Line('')
line.add(keyword, attr, v1,is_smooth=True,is_label_show=True,mark_line=[ "average"])
# line.add("商家B", attr, v2, is_smooth=True, mark_line=["max", "average"])
line

In [17]:
# 用户信息文件
os.path.exists('./users.torch.pkl')

True

In [18]:
# 预处理用户信息
userid_to_info = {}
if os.path.exists('./users.torch.pkl'):
    userid_to_info = torch.load('users.torch.pkl')
for x in statuses:
    userid_to_info[x['user']['id']] = x['user']
torch.save(userid_to_info, 'users.torch.pkl')
print(len(userid_to_info))

7198


In [19]:
# 发表人数/总消息数量
uni_num = len(set([x['user']['id'] for x in statuses ]))
print(uni_num, len(statuses), uni_num/len(statuses))

2673 10031 0.2664739308144751


In [20]:
# 发表top30的人
users_top = get_top([x['user']['id'] for x in statuses], num=30)
names_top = [ (userid_to_info[x[0]]['name'] ,  x[1], x[2],x[0]) for x in users_top]
pprint(names_top)
huabing(names_top)

[('糗百', 373, 0.03718472734522979, 'qiub'),
 ('L.1988', 285, 0.0284119230385804, '~q7mHGzyXZro'),
 ('大萌没有Ecup', 150, 0.014953643704516, 'Emikooo'),
 ('熊仔饼啊', 93, 0.009271259096799921, 'chowhong'),
 ('寒小梦', 83, 0.008274349516498854, 'cincylan'),
 ('那.谁', 80, 0.007975276642408534, '海边的暴走族'),
 ('李时珍的皮', 58, 0.005782075565746187, '闻言文'),
 ('安达充的走狗', 56, 0.005582693649685973, 'wylwddjz'),
 ('rinex20', 52, 0.005183929817565547, 'rinex'),
 ('越关山', 51, 0.00508423885953544, '~a8HE2eJqgxU'),
 ('乱耳朵', 51, 0.00508423885953544, 'Uncle_Luan'),
 ('逸博', 49, 0.004884856943475227, 'king66lele'),
 ('万能的饭否', 47, 0.0046854750274150135, 'wndff'),
 ('墨滞', 47, 0.0046854750274150135, 'subarasii'),
 ('马克先森', 45, 0.0044860931113548, 'talk2u'),
 ('云飞与小白', 45, 0.0044860931113548, 'adam1992'),
 ('MDJsoftware', 40, 0.003987638321204267, 'mdjsoftware'),
 ('轻轻河边操', 39, 0.00388794736317416, 'nicetomeetyou'),
 ('Talkaholic_L', 39, 0.00388794736317416, 'Talkaholic_L'),
 ('子浑月阿', 39, 0.00388794736317416, 'emmahathway'),
 (

In [21]:
def show_user_info(x):
    user = userid_to_info[x]
    pprint(user)
    return Image(url = user['profile_image_url_large'])

In [22]:
show_user_info(users_top[0][0])

{'birthday': '',
 'created_at': 'Wed Feb 17 04:46:39 +0000 2016',
 'description': '糗百非官方机器人，抓取自糗百首页的热门糗事。',
 'favourites_count': 0,
 'followers_count': 6555,
 'following': False,
 'friends_count': 0,
 'gender': '',
 'id': 'qiub',
 'location': '',
 'name': '糗百',
 'notifications': False,
 'photo_count': 143757,
 'profile_background_color': '#acdae5',
 'profile_background_image_url': 'http://static.fanfou.com/img/bg/0.png',
 'profile_background_tile': False,
 'profile_image_url': 'http://s3.meituan.net/v1/mss_3d027b52ec5a4d589e68050845611e68/avatar/s0/00/qa/f6.jpg?1455684943',
 'profile_image_url_large': 'http://s3.meituan.net/v1/mss_3d027b52ec5a4d589e68050845611e68/avatar/l0/00/qa/f6.jpg?1455684943',
 'profile_link_color': '#0066cc',
 'profile_sidebar_border_color': '#b2d1a3',
 'profile_sidebar_fill_color': '#e2f2da',
 'profile_text_color': '#222222',
 'protected': False,
 'screen_name': '糗百',
 'statuses_count': 437025,
 'unique_id': '~vywepXCNPTE',
 'url': 'http://www.qiushibaike.com/',

In [23]:
# 被引用top20的消息

repost_top = get_top([x['repost_status']['id'] for x in statuses if 'repost_status' in x], num=20)
for i in range(len(repost_top)):
    x = repost_top[i]
    for y in statuses:
        if 'repost_status' in y and y['repost_status']['id'] == x[0]:
            repost_top[i] = ('id: '+y['repost_status']['user']['id'] + ' name: '+y['repost_status']['user']['name']  +' '+ html.unescape(y['repost_status']['text']),x[0], x[1],x[2])
            break
            
pprint(repost_top)

[('id: karl name: 伊丽莎白桑坂田阙日 从微信取名看性格... 我觉得我是最后一个 微信名字越长 SB几率越高…………………………🙃️',
  'jVjDVOcc-Nc',
  59,
  0.024170421958213846),
 ('id: zenchan name: 布單 '
  '7年前张小龙的两条饭否，如今微信7.0新版本发布，开屏BGM用了Cohen的音乐，Slogan解读了王阳明的文字。',
  'EbTQCYB9vo4',
  55,
  0.022531749283080705),
 ('id: Olittle name: Olittle '
  '状态不佳，给学生上课使我快乐。转发抽奖，奖品为意大利语初级教程10节，一对一，我教。前100名转发+微信兑奖都算中奖，就是这么任性。有效期至2019年底，不学作废。上课需要提前预约一下，不然我可能在睡觉。上次抽中一个月免费的同学这次就不算了哟。',
  'b82ML9lAsU8',
  51,
  0.020893076607947564),
 ('id: ~ga9mK1MYxq4 name: 粉色很好啊但 '
  '我最好的朋友的爸爸出车祸面临截肢，现在她家经济来源全断了，每天要支出几万块的治疗费，她还在读大二，求求大家帮着捐一点或者转一下，求求了，谢谢谢谢谢谢！！！！！！！',
  '6B0fQ7TwVOA',
  43,
  0.017615731257681278),
 ('id: shiziwang name: 王金牙 '
  '又一年了。岁月风驰，变幻匆匆，不问人间值得，活着便已欢喜。新年，愿山海相逢，明朗可期，逝者不被消费，新生获得珍惜，你我聪慧独立，对流量微信，对压力陌陌，日子免于强咚，上马总有青云。2019，愿你情相随，家明亮，心有光，外卖和快递，诗篇与爱人，都会穿越寒夜和晨曦奔向你。',
  'gHSNQYH4xVA',
  41,
  0.016796394920114706),
 ('id: dyxin name: 饭而不否 '
  '马桶mt、多闪都已经被微信封杀了，如果今晚的聊天宝（子弹短信）不被微信封杀，那就不给罗永浩面子了。毕竟“斩尽杀绝，是对一个文明最高的重视”。',
  'V4Wfhj9futw',
  27,
  0.01106104055714

In [24]:
# 被引用top20的人
repost_user_id_top = get_top([x['repost_status']['user']['id'] for x in statuses if 'repost_status' in x], num=20)
repost_name_top =  [ (userid_to_info[x[0]]['name'] , x[1], x[2],  x[0]) for x in repost_user_id_top]
pprint(repost_name_top)

[('即刻精选', 83, 0.0340024580090127, 'jike_bot'),
 ('伊丽莎白桑坂田阙日', 63, 0.02580909463334699, 'karl'),
 ('粉色很好啊但', 61, 0.02498975829578042, '~ga9mK1MYxq4'),
 ('布單', 57, 0.023351085620647277, 'zenchan'),
 ('Olittle', 53, 0.021712412945514133, 'Olittle'),
 ('饭而不否', 49, 0.02007374027038099, 'dyxin'),
 ('王金牙', 43, 0.017615731257681278, 'shiziwang'),
 ('越关山', 41, 0.016796394920114706, '~a8HE2eJqgxU'),
 ('一介大叔', 37, 0.015157722244981565, 'ningqun'),
 ('安达充的走狗', 36, 0.014748054076198279, 'wylwddjz'),
 ('大萌没有Ecup', 36, 0.014748054076198279, 'Emikooo'),
 ('李时珍的皮', 35, 0.014338385907414994, '闻言文'),
 ('寒小梦', 30, 0.012290045063498567, 'cincylan'),
 ('阿因', 25, 0.010241704219582138, 'yin2000'),
 ('饭友经典语录', 25, 0.010241704219582138, 'fyjdyl'),
 ('马冇冇', 24, 0.009832036050798854, 'xyz_9900'),
 ('毒艹', 23, 0.009422367882015567, 'bisons'),
 ('果照', 23, 0.009422367882015567, '~q3PAKFOIRO8'),
 ('云飞与小白', 19, 0.0077836952068824255, 'adam1992'),
 ('无聊图4号机', 19, 0.0077836952068824255, '~KgjnhzJdzco')]


In [25]:
# top20 gender
gender_top = get_top([x['user']['gender'] or '未知' for x in statuses ], num=20)
pprint(gender_top)
huabing(gender_top)

[('男', 3846, 0.38341142458379024),
 ('未知', 3806, 0.379423786262586),
 ('女', 2379, 0.23716478915362377)]


In [29]:
# converter = {
#     'Jiangsu Sheng' : '江苏',
#     'Shandong Sheng' : '山东',
#     'Shanghai Shi' : '上海',
#     'Beijing' : '北京',
#     'Fujian': '福建',
#     'Guangxi Zhuangzu Zizhiqu': '广西',
#     'Yunnan': '云南',
#     'Tianjin Shi': '天津',
#     'Zhejiang Sheng' : '浙江',
#     'Guangdong': '广东',
#     'Sichuan': '四川',
#     'Hebei': '河北',
#     'TW': '台湾',
#     'Liaoning': '辽宁',
#     'Hunan': '湖南',
#     'Changchun Shi': '长春',
#     'KR' : '韩国',
#     'Henan Sheng': '河南',
#     'Shanxi Sheng': '山西',
#     'Jilin Sheng': '吉林',
#     'Anhui Sheng': '安徽',
#     'Heilongjiang Sheng': '黑龙江',
#     'Jiangxi Sheng': '江西',
#     'ES': '西班牙',
#     'Hubei': '湖北',
#     'SJ' : '斯瓦尔巴群岛和扬马延岛',
#     'Shaanxi': '陕西',
#     'Gansu Sheng': '甘肃',
#     'HK': '香港',
#     'CA' : '加拿大',
#     'Xinjiang Uygur Zizhiqu': '新疆',
#     'JP' : '日本',
# }
# torch.save(converter, 'geocoverter-en-cn.torch.pkl')

In [27]:
# top20 location

converter = torch.load('geocoverter-en-cn.torch.pkl')
def geotransform(x):
    if len(x) == 0:
        return '未知'
    if ',' in x and len(x.split(',')) == 2:
        try:
            a = x.split(',')
            results = rg.search((float(a[0]),float(a[1]))) # default mode = 2
            if len(results) == 0:
                return '未知'
            if results[0]['cc'] == 'CN':
                if results[0]['admin1'] in converter:
                    return converter[results[0]['admin1']]
                print('\n',len(results[0]['admin1']),results[0],'\n')
                return results[0]['admin1']
            else:
                if results[0]['cc'] in converter:
                    return converter[results[0]['cc']]
                print('\n',len(results[0]['cc']),results[0],'\n')
                return results[0]['cc']
        except Exception as e:
            print(x)
            return '未知'
    if ' ' in x and len(x.split(' ')) == 2:
        return x.split(' ')[0]
    return x
    
location_top = get_top([geotransform(x['location']) for x in tqdm(statuses)], num=20)
pprint(location_top)
huabing(location_top)

  0%|          | 0/10031 [00:00<?, ?it/s]

Loading formatted geocoded file...


  4%|▎         | 354/10031 [00:02<01:00, 160.61it/s]


 2 OrderedDict([('lat', '35.82255'), ('lon', '139.83905'), ('name', 'Yashio-shi'), ('admin1', 'Saitama'), ('admin2', ''), ('cc', 'JP')]) 



 64%|██████▎   | 6372/10031 [00:04<00:08, 431.89it/s]


 22 OrderedDict([('lat', '41.76055'), ('lon', '86.15231'), ('name', 'Sayibage'), ('admin1', 'Xinjiang Uygur Zizhiqu'), ('admin2', ''), ('cc', 'CN')]) 



100%|██████████| 10031/10031 [00:05<00:00, 1701.97it/s]


 2 OrderedDict([('lat', '35.82255'), ('lon', '139.83905'), ('name', 'Yashio-shi'), ('admin1', 'Saitama'), ('admin2', ''), ('cc', 'JP')]) 

[('未知', 3702, 0.3690559266274549),
 ('北京', 1042, 0.10387797826737115),
 ('广东', 679, 0.06769016050244243),
 ('上海', 625, 0.06230684876881667),
 ('辽宁', 442, 0.04406340344930715),
 ('浙江', 347, 0.03459276243644702),
 ('海外', 251, 0.025022430465556775),
 ('江苏', 223, 0.022231083640713786),
 ('湖北', 204, 0.02033695543814176),
 ('四川', 178, 0.017744990529358986),
 ('山东', 161, 0.016050244242847175),
 ('福建', 155, 0.015452098494666534),
 ('萌萌星', 150, 0.014953643704516),
 ('河南', 101, 0.010068786761040774),
 ('黑龙江', 89, 0.008872495264679493),
 ('陕西', 73, 0.007277439936197787),
 ('天津', 70, 0.006978367062107467),
 ('重庆', 70, 0.006978367062107467),
 ('湖南', 56, 0.005582693649685973),
 ('河北', 55, 0.005483002691655867)]


In [30]:
def remove_atag(s):
    if '<a' not in s:
        return s
    start = s.find('>')
    end = s.find('</a>',start)
    return s[start+1:end]

In [31]:
# top20 source
source_top = get_top([remove_atag(x['source']) for x in statuses ], num=20)
pprint(source_top)
huabing(source_top)

[('御飯 iOS', 1921, 0.19150633037583492),
 ('网页', 1755, 0.1749576313428372),
 ('猫饭', 1384, 0.13797228591366764),
 ('有饭', 747, 0.07446914564848968),
 ('饭团fantuan', 726, 0.07237563552985744),
 ('Android客户端', 497, 0.04954640614096301),
 ('蘑菇饭', 464, 0.0462566045259695),
 ('尚饭', 385, 0.03838101884159107),
 ('小饭', 383, 0.03818163692553085),
 ('糗事统计', 379, 0.03778287309341043),
 ('泡饭', 227, 0.022629847472834214),
 ('炒饭', 214, 0.021333865018442828),
 ('手机上网', 197, 0.019639118731931013),
 ('Twidere', 182, 0.018143754361479414),
 ('冷饭', 124, 0.012361678795733227),
 ('「御飯 iPhone」', 118, 0.011763533047552586),
 ('卐能的饭否', 47, 0.0046854750274150135),
 ('即刻精选', 33, 0.0032898016149935203),
 ('小跟班', 26, 0.0025919649087827734),
 ('Mr.Greeting', 24, 0.0023925829927225603)]


In [32]:
# nlp part

In [33]:
def myfilter(s):
    s = s.replace('<strong>','')
    s = s.replace('</strong>','')
    while '转@' in s:
        pos = s.find('转@')
        end = len(s)
        for endsym in [' ', ']', '：',':','@']:
            end2 = s.find(endsym,pos+2)
            if end2 == -1:
                end2 = len(s)
            end = min(end,end2)
        if end-pos > 15:
            print('alert [转@] too long',s)
            s = s[:pos] + s[pos+2:]
        else:
            s = s[:pos] + s[end:]
    while '@' in s:
        pos = s.find('@')
        end = len(s)
        for endsym in [' ', ']', '：',':','@', '\n']:
            end2 = s.find(endsym,pos+1)
            if end2 == -1:
                end2 = len(s)
            end = min(end,end2)
        if end-pos > 15:
            print('alert [@] too long',s)
            s = s[:pos] + s[pos+1:]
        else:
            s = s[:pos] + s[end:]
    if '@' in s:
        print('alert @ in ', s)
    return s

In [34]:
sentences = []
for x in tqdm(statuses):
    s = html.unescape(x['text']) 
    s = myfilter(s)
    if keyword not in s:
        continue
    if '</' in s:
        print(s)
    ss = SentenceSplitter.split(s)
    for t in ss:
        if len(t) > 4:
            sentences.append(t)
    

 59%|█████▉    | 5954/10031 [00:00<00:00, 12192.58it/s]

alert [@] too long (1/2) @sein首先这个确实很反人类，强烈抵制，但是他给的地址真的是能打开的，但是只能用微信打开，而且可以开票地址如下：http://fan.fo/XCqu6
点击<a href="https://support.weixin.qq.com/cgi-bin/mmsupport-bin/getopendays?from=groupmessage&amp;isappinstalled=0">这里</a>，看看今天是你和微信同行的第几天？
转


 79%|███████▉  | 7951/10031 [00:00<00:00, 8497.64it/s] 

alert [@] too long 决定2019年在社交网络上装个死，朋友圈呀微博呀豆瓣广播呀。所有群里除非工作需要，不@了也可以不出现。本来想直接废了现在的微信账号只用跟家人密友的小号的，但接翻译办签证还是得用到。
alert [@] too long 以前的人有名有字有号，现在大家都只有一个名字了。出国之后即使是同一个实验室的中国人，有的人我只知道他们名字怎么读，具体是哪个字已经不确定了。平常微信里要不就是长按自动@他们的昵称，要不就是直接用拼音。这么想想有点可怕。


100%|██████████| 10031/10031 [00:01<00:00, 9482.61it/s]


In [35]:
print(len(sentences))

20334


In [36]:
# 对sentences去重。。 拒绝转发的影响 （存疑？）
# 不去重

In [37]:
# sentences = list(set(sentences))

In [38]:
print(len(list(set(sentences))))

16113


In [39]:
# torch.save(['尼玛', '卧槽', '傻逼', '对了', '有木有', '他妈'],'blacklist.torch.pkl' )
def add_to_blacklist(w):
    global blacklist
    blacklist = torch.load('blacklist.torch.pkl')
    blacklist.append(w)
    torch.save(blacklist,'blacklist.torch.pkl')

In [40]:
# 无意义
blacklist = torch.load('blacklist.torch.pkl')

In [41]:
words_all = []

segmentor = Segmentor()
postagger = Postagger()
recognizer = NamedEntityRecognizer()
postagger.load('./ltp_data_v3.4.0/pos.model')
segmentor.load('./ltp_data_v3.4.0/cws.model')
recognizer.load('./ltp_data_v3.4.0/ner.model')

#todo 情感分析

for s in tqdm(sentences):
    words = segmentor.segment(s)
    postags = postagger.postag(words) #词性标注
    netags = recognizer.recognize(words, postags) # 命名实体识别
    for i in range(len(words)):
        if words[i] in blacklist:
            continue
        words_all.append((words[i],postags[i],netags[i]))

segmentor.release()
postagger.release()
recognizer.release()

100%|██████████| 20334/20334 [00:26<00:00, 756.72it/s]


In [42]:
print(len(words_all))

393126


In [43]:
get_top([x[0] for x in words_all if len(x[0])>1 and x[1]!='wp'], num = 25)

[('微信', 10907, 0.06922134710948359),
 ('一个', 1855, 0.011772769678930232),
 ('没有', 1177, 0.007469838227547646),
 ('可以', 862, 0.0054706886594274185),
 ('自己', 861, 0.005464342152861957),
 ('现在', 823, 0.0052231749033744375),
 ('什么', 814, 0.005166056344285288),
 ('这个', 801, 0.005083551758934294),
 ('朋友', 775, 0.004918542588232307),
 ('手机', 736, 0.004671028832179327),
 ('知道', 713, 0.004525059181173723),
 ('今天', 702, 0.004455247608953651),
 ('时候', 660, 0.004188694333204288),
 ('觉得', 600, 0.0038079039392766253),
 ('然后', 600, 0.0038079039392766253),
 ('还是', 577, 0.003661934288271021),
 ('怎么', 538, 0.0034144205322180405),
 ('好友', 518, 0.0032874904009088197),
 ('一下', 515, 0.003268450881212437),
 ('喜欢', 514, 0.0032621043746469754),
 ('我们', 514, 0.0032621043746469754),
 ('朋友圈', 509, 0.0032303718418196705),
 ('发现', 497, 0.003154213763034138),
 ('看到', 487, 0.0030907486973795273),
 ('饭否', 477, 0.003027283631724917)]

In [44]:
posdict = {}
nedict = {}
for x in tqdm(words_all):
    if x[1] not in posdict:
        posdict[x[1]] = []
    if x[2] not in nedict:
        nedict[x[2]] = []
    posdict[x[1]].append(x)
    nedict[x[2]].append(x)

100%|██████████| 393126/393126 [00:00<00:00, 954595.48it/s]


In [45]:
# 词性
print(posdict.keys())


dict_keys(['nt', 'n', 'v', 'wp', 'a', 'u', 'r', 'm', 'q', 'b', 'c', 'nd', 'd', 'p', 'ws', 'ns', 'nh', 'nl', 'o', 'z', 'j', 'e', 'i', 'nz', 'k', 'ni', 'h', '%'])


In [46]:
# ['i'] 成语
pprint(get_top(posdict['i'], num=25))

[('一对一', 54, 0.02854122621564482),
 ('没什么', 49, 0.025898520084566595),
 ('差不多', 45, 0.023784355179704016),
 ('风驰', 43, 0.022727272727272728),
 ('有意思', 38, 0.0200845665961945),
 ('家明亮', 34, 0.017970401691331923),
 ('斩尽杀绝', 28, 0.014799154334038054),
 ('不怎么', 25, 0.013213530655391121),
 ('莫名其妙', 24, 0.012684989429175475),
 ('他妈的', 23, 0.012156448202959831),
 ('是的', 19, 0.01004228329809725),
 ('你好', 16, 0.008456659619450317),
 ('晚安', 15, 0.007928118393234672),
 ('好不容易', 14, 0.007399577167019027),
 ('呃', 13, 0.006871035940803382),
 ('没关系', 12, 0.006342494714587738),
 ('一模一样', 12, 0.006342494714587738),
 ('群聊', 11, 0.005813953488372093),
 ('无所谓', 11, 0.005813953488372093),
 ('微信', 11, 0.005813953488372093),
 ('不知不觉', 11, 0.005813953488372093),
 ('事实上', 11, 0.005813953488372093),
 ('说实话', 11, 0.005813953488372093),
 ('天定', 10, 0.005285412262156448),
 ('举例来说', 10, 0.005285412262156448)]


In [47]:

# ['j'] 缩略词
pprint(get_top(posdict['j'], num=25))


[('微博', 54, 0.09800362976406533),
 ('春晚', 28, 0.050816696914700546),
 ('哈', 16, 0.029038112522686024),
 ('加', 15, 0.02722323049001815),
 ('统战', 13, 0.023593466424682397),
 ('美', 13, 0.023593466424682397),
 ('清华', 12, 0.021778584392014518),
 ('糗百', 11, 0.019963702359346643),
 ('九运司', 9, 0.016333938294010888),
 ('春运', 9, 0.016333938294010888),
 ('人大', 9, 0.016333938294010888),
 ('哥', 9, 0.016333938294010888),
 ('华大', 8, 0.014519056261343012),
 ('社保', 7, 0.012704174228675136),
 ('宣发', 6, 0.010889292196007259),
 ('欧', 6, 0.010889292196007259),
 ('咪蒙', 6, 0.010889292196007259),
 ('阿', 6, 0.010889292196007259),
 ('北大', 6, 0.010889292196007259),
 ('个税', 5, 0.009074410163339383),
 ('央行', 5, 0.009074410163339383),
 ('蒙', 5, 0.009074410163339383),
 ('银联', 5, 0.009074410163339383),
 ('人均', 5, 0.009074410163339383),
 ('社恐', 5, 0.009074410163339383)]


In [48]:

# ['n'] 名词
pprint(get_top(posdict['n'], num=25))


[('微信', 10417, 0.15738978031607892),
 ('人', 2027, 0.030625812105279062),
 ('朋友', 775, 0.011709424953917746),
 ('手机', 736, 0.011120176472365757),
 ('时候', 660, 0.009971897380110598),
 ('号', 555, 0.008385459160547547),
 ('钱', 545, 0.008234369806303448),
 ('好友', 517, 0.007811319614419968),
 ('朋友圈', 509, 0.007690448131024688),
 ('消息', 438, 0.0066177137158915786),
 ('群', 395, 0.005968029492641949),
 ('红包', 388, 0.005862266944671079),
 ('公众', 377, 0.005696068655002568),
 ('电话', 370, 0.005590306107031698),
 ('视频', 364, 0.005499652494485238),
 ('头像', 338, 0.005106820173450578),
 ('功能', 329, 0.0049708397546308885),
 ('饭否', 326, 0.004925512948357659),
 ('妈', 318, 0.004804641464962379),
 ('事', 317, 0.004789532529537969),
 ('微信群', 305, 0.004608225304445049),
 ('时间', 288, 0.004351373402230079),
 ('社交', 280, 0.004230501918834799),
 ('同学', 277, 0.004185175112561569),
 ('话', 276, 0.004170066177137159)]


In [49]:

# ['nh'] 人名词
pprint(get_top(posdict['nh'], num=25))


[('张', 108, 0.06112054329371817),
 ('小龙', 86, 0.04867006225240521),
 ('张小龙', 80, 0.045274476513865305),
 ('王阳明', 62, 0.03508771929824561),
 ('王欣', 42, 0.023769100169779286),
 ('罗永浩', 37, 0.020939445387662705),
 ('汪', 26, 0.014714204867006225),
 ('黄', 25, 0.014148273910582909),
 ('刘慈欣', 18, 0.010186757215619695),
 ('玛德', 17, 0.009620826259196379),
 ('咪蒙', 14, 0.00792303338992643),
 ('王强', 13, 0.007357102433503113),
 ('杨', 13, 0.007357102433503113),
 ('曹操', 13, 0.007357102433503113),
 ('马', 11, 0.00622524052065648),
 ('ὢ･᷅', 11, 0.00622524052065648),
 ('･᷄', 11, 0.00622524052065648),
 ('郭靖', 10, 0.005659309564233163),
 ('王', 10, 0.005659309564233163),
 ('小王', 9, 0.0050933786078098476),
 ('赵', 9, 0.0050933786078098476),
 ('+黄蓉', 9, 0.0050933786078098476),
 ('化腾', 9, 0.0050933786078098476),
 ('王小剑', 8, 0.004527447651386531),
 ('陈', 8, 0.004527447651386531)]


In [50]:

# ['ni'] 组织名
pprint(get_top(posdict['ni'], num=25))


[('教育部', 11, 0.34375),
 ('清华大学', 6, 0.1875),
 ('人事部', 5, 0.15625),
 ('微软', 3, 0.09375),
 ('中国银行', 2, 0.0625),
 ('外交部', 1, 0.03125),
 ('组队', 1, 0.03125),
 ('公安部', 1, 0.03125),
 ('腾讯', 1, 0.03125),
 ('悦己', 1, 0.03125)]


In [51]:

# ['nz'] other proper name
pprint(get_top(posdict['nz'], num=25))


[('微信', 257, 0.23664825046040516),
 ('腾讯', 129, 0.11878453038674033),
 ('淘宝', 69, 0.06353591160220995),
 ('百度', 59, 0.05432780847145488),
 ('安卓', 50, 0.04604051565377532),
 ('英文', 33, 0.03038674033149171),
 ('搜狗', 28, 0.02578268876611418),
 ('华为', 25, 0.02302025782688766),
 ('英语', 24, 0.022099447513812154),
 ('豆瓣', 23, 0.02117863720073665),
 ('中文', 21, 0.019337016574585635),
 ('京东', 21, 0.019337016574585635),
 ('美团', 14, 0.01289134438305709),
 ('谷歌', 13, 0.011970534069981584),
 ('人民日报', 11, 0.010128913443830571),
 ('摇一摇', 10, 0.009208103130755065),
 ('诺基亚', 10, 0.009208103130755065),
 ('肯德基', 10, 0.009208103130755065),
 ('魅族', 9, 0.008287292817679558),
 ('麦当劳', 6, 0.0055248618784530384),
 ('三星', 6, 0.0055248618784530384),
 ('新浪', 6, 0.0055248618784530384),
 ('多益', 5, 0.004604051565377533),
 ('广汇', 5, 0.004604051565377533),
 ('嘉嘉', 5, 0.004604051565377533)]


In [52]:

# ['v'] verb len >= 2
# pprint(get_top([x for x in posdict['v'] if len(x[0])>=2], num=25))


In [53]:

# foreign words	
pprint(get_top([x for x in posdict['ws'] if len(x[0])>=4 and not x[0].startswith('http://')], num=25))


[('Ecup', 62, 0.038461538461538464),
 ('Cohen', 58, 0.03598014888337469),
 ('Slogan', 56, 0.034739454094292806),
 ('iPhone', 47, 0.02915632754342432),
 ('gelei669', 45, 0.02791563275434243),
 ('kindle', 37, 0.02295285359801489),
 ('Google', 20, 0.01240694789081886),
 ('Telegram', 19, 0.011786600496277916),
 ('iPad', 18, 0.011166253101736972),
 ('hhhh', 18, 0.011166253101736972),
 ('ipad', 16, 0.009925558312655087),
 ('wifi', 14, 0.008684863523573201),
 ('Android', 14, 0.008684863523573201),
 ('Tony', 12, 0.007444168734491315),
 ('WhatsApp', 12, 0.007444168734491315),
 ('Facebook', 9, 0.005583126550868486),
 ('CSDN', 9, 0.005583126550868486),
 ('iMessage', 9, 0.005583126550868486),
 ('Store', 8, 0.004962779156327543),
 ('google', 8, 0.004962779156327543),
 ('facebook', 8, 0.004962779156327543),
 ('speak', 8, 0.004962779156327543),
 ('hhhhh', 7, 0.004342431761786601),
 ('ha-sir', 7, 0.004342431761786601),
 ('Instagram', 7, 0.004342431761786601)]


In [54]:
# 命名实体
print(nedict.keys())


dict_keys(['O', 'B-Ni', 'I-Ni', 'E-Ni', 'S-Nh', 'B-Ns', 'I-Ns', 'E-Ns', 'S-Ns', 'B-Nh', 'E-Nh', 'S-Ni'])


In [55]:


# 人名
pprint(get_top(nedict['S-Nh'], num=25))


[('张小龙', 80, 0.05606166783461808),
 ('王阳明', 62, 0.043447792571829014),
 ('王欣', 42, 0.02943237561317449),
 ('罗永浩', 37, 0.02592852137351086),
 ('汪', 26, 0.018220042046250877),
 ('黄', 23, 0.0161177295024527),
 ('张', 23, 0.0161177295024527),
 ('刘慈欣', 18, 0.012613875262789068),
 ('玛德', 17, 0.011913104414856343),
 ('马', 15, 0.01051156271899089),
 ('咪蒙', 14, 0.009810791871058164),
 ('王强', 13, 0.009110021023125438),
 ('杨', 12, 0.008409250175192713),
 ('王', 10, 0.00700770847932726),
 ('小王', 9, 0.006306937631394534),
 ('赵', 9, 0.006306937631394534),
 ('王小剑', 8, 0.005606166783461808),
 ('老公', 8, 0.005606166783461808),
 ('小宝', 7, 0.004905395935529082),
 ('曹操', 7, 0.004905395935529082),
 ('马化腾', 7, 0.004905395935529082),
 ('陈', 7, 0.004905395935529082),
 ('😂', 7, 0.004905395935529082),
 ('单小龙', 6, 0.004204625087596356),
 ('秦晖', 6, 0.004204625087596356)]


In [56]:

# 机构名
pprint(get_top(nedict['S-Ni'], num=25))


[('教育部', 11, 0.3548387096774194),
 ('清华大学', 6, 0.1935483870967742),
 ('人事部', 5, 0.16129032258064516),
 ('微软', 3, 0.0967741935483871),
 ('中国银行', 2, 0.06451612903225806),
 ('外交部', 1, 0.03225806451612903),
 ('组队', 1, 0.03225806451612903),
 ('公安部', 1, 0.03225806451612903),
 ('悦己', 1, 0.03225806451612903)]


In [57]:

# 地名
pprint(get_top(nedict['S-Ns'], num=25))

[('中国', 122, 0.11296296296296296),
 ('意大利', 57, 0.05277777777777778),
 ('北京', 53, 0.049074074074074076),
 ('上海', 50, 0.046296296296296294),
 ('日本', 28, 0.025925925925925925),
 ('台湾', 24, 0.022222222222222223),
 ('美国', 24, 0.022222222222222223),
 ('京东', 21, 0.019444444444444445),
 ('阿里', 20, 0.018518518518518517),
 ('杭州', 18, 0.016666666666666666),
 ('美', 14, 0.012962962962962963),
 ('广州', 14, 0.012962962962962963),
 ('西安', 13, 0.012037037037037037),
 ('深圳', 13, 0.012037037037037037),
 ('咪蒙', 12, 0.011111111111111112),
 ('安阳', 12, 0.011111111111111112),
 ('苏州', 11, 0.010185185185185186),
 ('宁波', 11, 0.010185185185185186),
 ('武汉', 11, 0.010185185185185186),
 ('韩国', 11, 0.010185185185185186),
 ('加拿大', 10, 0.009259259259259259),
 ('加', 10, 0.009259259259259259),
 ('香港', 10, 0.009259259259259259),
 ('山东', 10, 0.009259259259259259),
 ('成都', 10, 0.009259259259259259)]


In [66]:
# 以上内容均为自动生成...